In [1]:
#Jason Galvan

In [2]:
#!pip install webhoseio
!pip install simhash

In [3]:
import webhoseio, os
from gensim.models import KeyedVectors
import json
from simhash import Simhash, SimhashIndex
import numpy as np

In [4]:
#Importing Saved Webhose Dataset

json_data=open('/Users/batman/Downloads/webhose_IBM.json').readlines()
newsfeeds_read = []
for line in json_data:
    newsfeeds_read.append(json.loads(line))
print(len(newsfeeds_read))

3779


In [5]:
# Printing Original Dataset by Ids/Titles Only ---placed # to prevent lengthly output 
feeds = []
i = 0
for feed in newsfeeds_read:
    feed['id'] = i
    #print(feed['id'], (feed['title']))  #Can unhash and code will print id + titles separate from remaining data 
    i += 1
    feeds.append(feed)

In [6]:
model_path = '/Users/batman/Downloads/'
def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_w2v_AP    = load_wordvec_model('Word2Vec Google News', 'GoogleNews-vectors-negative300.bin.gz', True)
#model_fasttext = load_wordvec_model('FastText', 'fastText_wiki_en.vec', False)

Loading Word2Vec Google News model...
Finished loading Word2Vec Google News model...


In [7]:
# Function checks whether the input words are present in the vocabulary for the model
def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

In [8]:
# Function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    #if len(s1words) < 1 | len(s2words) < 1:
    #return 0
    
    try:
        
        output = vectors.n_similarity(s1words, s2words)
    
    except:
        
        output = 0
        
    return output

In [9]:
def cleanup(input):
    # remove English stopwords
    input = input.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
    input = re.sub(r'[^a-zA-Z0-9 ]', '', input)
    return input

In [10]:
#Set distance to 5 because only using Simhash algorithm.  Simhashing requires a smaller distance because it 
#operates by splitting/hashing text into chunks.  If using w2vec, I would set distance higher--at approximately 20.  
#W2vec rquires higher distance because it employs a vector method for comparison purposes, i.e. vectorizing words.  

#I used several distance amounts and found that 5 produced the most accurate results. 

import logging
logging.getLogger('simhash').setLevel(logging.CRITICAL) 

distance = 5
objs = [(str(feed['id']), Simhash(str(feed['title']))) for feed in feeds]
index = SimhashIndex(objs, k=distance)

In [11]:
#Designed Simhash alogrithm which saves duplicate ids/titles under variable dup_indices

dup_indices = []
dict1= dict()

for id in range(len(feeds)):
    dict1[id] = [id]
    
for j in range(len(feeds)):
    feed_sel = feeds[j]
    feed_hash = Simhash(str(feed_sel['title']))
    indices = index.get_near_dups(feed_hash)
    
    for a in indices: 
        if (a not in dup_indices) and (a != j) and (j not in dict1[int(a)]):
            dup_indices.append(a)
        dict1[a] = dict1[int(a)].append(int(j))
        


In [12]:
#Length of feeds ='s the total length of the entire Dataset with ALL ids/titles.  I am using this as a benchmark.  

len(feeds) 

3779

In [13]:
#This variable ='s total amount of Duplicates identified by the Simhash Algorithm.  Our goal is to remove
#the duplicates from the original dataset (feeds).   

len(dup_indices)

1285

In [14]:
#Collecting Simhash Duplicate ids+ titles + text--Saved below as a Json file: 
dup_titles=[]

for dupi in dup_indices:
    dup_titles.append((feeds[int(dupi)]['id'], feeds[int(dupi)]['title'], feeds[int(dupi)]['text']))

In [15]:
#Collecting Simhash Duplicate ids + titles 'only' for Ease of Review--
#Saved below as a Json file: 

dup_titles2=[]

for dupi in dup_indices:
    dup_titles2.append((feeds[int(dupi)]['id'], feeds[int(dupi)]['title']))

In [40]:
len(dup_titles)

1285

In [41]:
len(dup_titles2)

1285

In [42]:
#Collecting Webhose 'ids' +'titles' + 'text' to variable orig_titles--Saved below as a 
#Json file:

orig_titles= []

for feed in newsfeeds_read:
    orig_titles.append((feed['id'], feed['title'], feed['text']))

In [43]:
#Collecting Webhose only 'ids' and 'titles' to variable orig_titles--for Ease of Review
#Saved below as a Json file:

orig_titles2= []

for feed in orig_titles:
    orig_titles2.append((feed['title']))
    

TypeError: tuple indices must be integers or slices, not str

In [30]:
#Ensuring length of orig_titles ='s length of feeds above (total titles in Webhose dataset)
len(orig_titles)

3779

In [31]:
len(orig_titles2)

3779

In [32]:
###Removing duplicate titles from original Webhose dataset =deduplicated subset of titles---
#Assignment Goal (leaves us with deduplicated ids + titles + text)

dedup_titles = (set(orig_titles).difference(dup_titles))

In [33]:
#Deduplicated ids + titles:  for ease of review 

dedup_titles2 = (set(orig_titles2).difference(dup_titles2))

In [34]:
#Ensuring duplicate titles were removed:  total dataset titles (3779) - duplicate titles 
#(1285) ='s 2494 titles
len(dedup_titles)

2494

In [35]:
len(dedup_titles2)

2903

In [26]:
#Sorting deduplicated subset or ids + titles + text--
#Saved below as a Json file:

sorted_dedupTitles = sorted(dedup_titles, reverse=False)

In [27]:
#Sorting deduplicated subset or ids + titles for ease of review--
#Saved below as a Json file:
sorted_dedupTitles2 = sorted(dedup_titles2, reverse=False)

In [28]:
#Ensuring sorted deduplicated subset is proper length (2494):

len(sorted_dedupTitles)

2494

In [135]:
len(sorted_dedupTitles2)

2494

In [145]:
#Saving all Original Webhose Ids/Titles to json file: 

with open ('/Users/batman/Downloads/originalTitles.json', "w") as data_file:
    for x in orig_titles2:
        line = json.dumps(x)
        data_file.write(line)
        data_file.write("\n")

In [146]:
#Saving Deduplicated Subset of Ids/Titles/Text to json file ---Non-Sorted: Assignment Goal

with open ('/Users/batman/Downloads/deduplicatedData.json', "w") as data_file:
    for x in dedup_titles:
        line = json.dumps(x)
        data_file.write(line)
        data_file.write("\n")

In [147]:
#Saving Deduplicated Subset of Ids/Titles (only) to json file ---Non-Sorted: 
with open ('/Users/batman/Downloads/deduplicatedTitles.json', "w") as data_file:
    for x in dedup_titles2:
        line = json.dumps(x)
        data_file.write(line)
        data_file.write("\n")

In [148]:
#Saving Sorted Deduplicated Subset of Ids/Titles/Text to json file---Sorted:  

with open ('/Users/batman/Downloads/sorted_deduplicatedData.json', "w") as data_file:
    for x in sorted_dedupTitles:
        line = json.dumps(x)
        data_file.write(line)
        data_file.write("\n")

In [149]:
#Saving Sorted Deduplicated Subset of Ids/Titles to json file---Sorted:
with open ('/Users/batman/Downloads/sorted_deduplicatedTitles.json', "w") as data_file:
    for x in sorted_dedupTitles2:
        line = json.dumps(x)
        data_file.write(line)
        data_file.write("\n")

In [150]:
#Saving Simhash Duplicate Ids/Titles/Text--for Reference.  I did not want to print above as it is 
#1285 articles....very long.  So decided to save into a json file for review.   

with open ('/Users/batman/Downloads/duplicateData.json', "w") as data_file:
    for j in dup_titles:
        line = json.dumps(j)
        data_file.write(line)
        data_file.write("\n")

In [151]:
##Saving Simhash Duplicate Ids/Titles--for Reference.

with open ('/Users/batman/Downloads/duplicateTitles.json', "w") as data_file:
    for j in dup_titles2:
        line = json.dumps(j)
        data_file.write(line)
        data_file.write("\n")

In [152]:
#Reading JSON file back into Python array of JSON objects and confirming total count

In [153]:
json_data=open('/Users/batman/Downloads/deduplicatedData.json').readlines()
dedup_subset = []
for line in json_data:
    dedup_subset.append(json.loads(line))
print(len(dedup_subset))

2494


In [23]:
#End of Assignment 7 